##### 版權 2024 Google LLC.


In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# PaLM API： 使用 Python 進行文本快速入門


<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/palm_docs/text_quickstart"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />檢視 on ai.google.dev</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/doggy8088/generative-ai-docs/blob/main/site/zh/palm_docs/text_quickstart.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />在 Google Colab 上執行</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/doggy8088/generative-ai-docs/blob/main/site/zh/palm_docs/text_quickstart.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />在 GitHub 上檢視原始碼</a>
  </td>
</table>


在此筆記本中，你將瞭解如何著手使用 PaLM API，它可讓你存取 Google 最新的大型語言模型。在此，你將瞭解如何使用 PaLM API 的文字生成功能。


## 設定

**注意：** 目前 PaLM API [僅在特定地區提供](https://developers.generativeai.google/available_regions)。


首先，下載和安裝 PaLM API Python 函式庫。


In [2]:
!pip install -q google-generativeai

In [3]:
import pprint
import google.generativeai as palm

### 取得 API 金鑰

**要** 開始使用，你**需要** [建立一個 API 金鑰](https://developers.generativeai.google/tutorials/setup)。


In [4]:
palm.configure(api_key='YOUR_API_KEY')

## 文字生成

使用`palm.list_models`函式尋找可用的模型：


In [5]:
models = [m for m in palm.list_models() if 'generateText' in m.supported_generation_methods]
model = models[0].name
print(model)

models/text-bison-001


使用 `palm.generate_text` 方法產生文字：


In [6]:
prompt = """
You are an expert at solving word problems.

Solve the following problem:

I have three houses, each with three cats.
each cat owns 4 mittens, and a hat. Each mitten was
knit from 7m of yarn, each hat from 4m.
How much yarn was needed to make all the items?

Think about it step by step, and show your work.
"""

completion = palm.generate_text(
    model=model,
    prompt=prompt,
    temperature=0,
    # The maximum length of the response
    max_output_tokens=800,
)

print(completion.result)

There are 3 houses * 3 cats / house = 9 cats. So, 9 cats * 4 mittens / cat = 36 mittens were made. Also, 9 cats * 1 hat / cat = 9 hats were made. So, 36 mittens * 7m / mitten = 252m of yarn was used for the mittens. Also, 9 hats * 4m / hat = 36m of yarn was used for the hats. In total, 252m + 36m = 288m of yarn was used.
Thus, the answer is 288.


## 其他選項

`palm.generate_text` 函式還有幾個其他值得一提的參數。


### 停止序列

使用 `stop_sequences` 參數可提早停止產生。

例如，LLM 經常在算術上出錯。你可以透過將方程式放入 `<calc>` 標籤中，要求模型「使用計算器」。

讓模型在關閉標籤時停止，以便你可以編輯提示：


In [7]:
calc_prompt = f"""
Please solve the following problem.

{prompt}

----------------

Important: Use the calculator for each step.
Don't do the arithmetic in your head. 

To use the calculator wrap an equation in <calc> tags like this: 

<calc> 3 cats * 2 hats/cat </calc> = 6

----------------

"""

equation=None
while equation is None:
    completion = palm.generate_text(
        model=model,
        prompt=calc_prompt,
        stop_sequences=['</calc>'],
        # The maximum length of the response
        max_output_tokens=800,
    )

    try:
        response, equation = completion.result.split('<calc>', maxsplit=1)
    except Exception:
        continue

In [8]:
print(response)

Chain-of-thought:

There are three houses, and each house has three cats, so there are 3 houses * 3 cats / house = 9 cats. Each cat has 4 mittens, so the cats need 9 cats * 4 mittens / cat = 36 mittens. Each mitten takes 7m of yarn, so 36 mittens * 7m / mitten = 252m of yarn. Each cat has a hat, and each hat takes 4m of yarn, so 9 cats * 4m / cat = 36m of yarn. So, in total, 36m + 252m = 288m of yarn were needed.

The answer should be 


In [9]:
print(equation)

 9 cats * 4 mittens / cat 


在這裡你可以計算結果，並組合新的提示訊息，供模型繼續使用。
對於一個完整的操作實作，請見 [文字計算器範例](../examples/text_calculator.ipynb)。


### 候選者

通常，LLM 生產的文字中會有一定程度的隨機性。(在 [LLM 簡介](https://developers.generativeai.google/guide/concepts)中閱讀更多原因)。這表示當你使用相同的輸入呼叫 API 超過一次時，你可能會得到不同的回應。你可以利用這項功能，取得另類的模型回應。

溫度引數會控制回應的差異。`palm.Model` 物件會提供 `temperature` 及其他參數的預設值。


In [10]:
models[0]

Model(name='models/text-bison-001', base_model_id='', version='001', display_name='Text Bison', description='Model targeted for text generation.', input_token_limit=8196, output_token_limit=1024, supported_generation_methods=['generateText'], temperature=0.7, top_p=0.95, top_k=40)

`candidate_count` 參數控制回傳的回應數量：


In [11]:
completion = palm.generate_text(
    model=model,
    prompt=prompt,
    # The number of candidates to return
    candidate_count=8,
    # Set the temperature to 1.0 for more variety of responses.
    temperature=1.0,
    max_output_tokens=800,
)

print(completion.result)

In each house there are 3 cats * 4 mittens / cat = 12 mittens. In total there are 3 houses * 12 mittens / house = 36 mittens. In total there are 36 mittens * 7m / mitten = 252m of yarn for the mittens. In total there are 3 houses * 3 cats / house * 1 hat / cat = 9 hats. In total there are 9 hats * 4m / hat = 36m of yarn for the hats. In total there are 36m yarn for the hats + 252m yarn for the mittens = 288m of yarn.
The answer: 288.


當你要求多個候選項時，`Completion.result` 屬性仍只包含第一個。 `Completion.candidates` 屬性包含所有候選項：


In [12]:
import pprint
pprint.pprint(completion.candidates)

[{'output': 'In each house there are 3 cats * 4 mittens / cat = 12 mittens. In '
            'total there are 3 houses * 12 mittens / house = 36 mittens. In '
            'total there are 36 mittens * 7m / mitten = 252m of yarn for the '
            'mittens. In total there are 3 houses * 3 cats / house * 1 hat / '
            'cat = 9 hats. In total there are 9 hats * 4m / hat = 36m of yarn '
            'for the hats. In total there are 36m yarn for the hats + 252m '
            'yarn for the mittens = 288m of yarn.\n'
            'The answer: 288.',
  'safety_ratings': [{'category': <HarmCategory.HARM_CATEGORY_DEROGATORY: 1>,
                      'probability': <HarmProbability.NEGLIGIBLE: 1>},
                     {'category': <HarmCategory.HARM_CATEGORY_TOXICITY: 2>,
                      'probability': <HarmProbability.NEGLIGIBLE: 1>},
                     {'category': <HarmCategory.HARM_CATEGORY_VIOLENCE: 3>,
                      'probability': <HarmProbability.NEGLIGIBLE: 1>}

因此，既然你知道這個問題的答案，就能輕鬆檢查解決率：


In [13]:
import numpy as np
np.mean(['288' in c['output'] for c in completion.candidates])

0.75